In [2]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder \
    .appName("SparkSQL") \
    .master("local[*]") \
    .getOrCreate()


In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('sales')


In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

spark.sql("""select round(avg(price), 2), date_built, any_value(bedrooms) from sales where bedrooms=4 group by date_built""").show()

+--------------------+----------+-------------------+
|round(avg(price), 2)|date_built|any_value(bedrooms)|
+--------------------+----------+-------------------+
|           296800.75|      2010|                  4|
|            302141.9|      2011|                  4|
|           298233.42|      2012|                  4|
|           299999.39|      2013|                  4|
|           299073.89|      2014|                  4|
|           307908.86|      2015|                  4|
|           296050.24|      2016|                  4|
|           296576.69|      2017|                  4|
+--------------------+----------+-------------------+



In [12]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

spark.sql("""select round(avg(price), 2), date_built, any_value(bedrooms), any_value(bathrooms) from sales where bedrooms=3 and bathrooms=3 group by date_built""").show()

+--------------------+----------+-------------------+--------------------+
|round(avg(price), 2)|date_built|any_value(bedrooms)|any_value(bathrooms)|
+--------------------+----------+-------------------+--------------------+
|           292859.62|      2010|                  3|                   3|
|           291117.47|      2011|                  3|                   3|
|           293683.19|      2012|                  3|                   3|
|           295962.27|      2013|                  3|                   3|
|           290852.27|      2014|                  3|                   3|
|            288770.3|      2015|                  3|                   3|
|           290555.07|      2016|                  3|                   3|
|           292676.79|      2017|                  3|                   3|
+--------------------+----------+-------------------+--------------------+



In [13]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

spark.sql("""select round(avg(price), 2), date_built, any_value(bedrooms), any_value(bathrooms), any_value(floors), any_value(sqft_living) from sales where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000 group by date_built""").show()

+--------------------+----------+-------------------+--------------------+-----------------+----------------------+
|round(avg(price), 2)|date_built|any_value(bedrooms)|any_value(bathrooms)|any_value(floors)|any_value(sqft_living)|
+--------------------+----------+-------------------+--------------------+-----------------+----------------------+
|           285010.22|      2010|                  3|                   3|                2|                  2049|
|           276553.81|      2011|                  3|                   3|                2|                  2471|
|           307539.97|      2012|                  3|                   3|                2|                  2377|
|           303676.79|      2013|                  3|                   3|                2|                  2045|
|           298264.72|      2014|                  3|                   3|                2|                  2271|
|           297609.97|      2015|                  3|                   

In [41]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select * from (select round(avg(price), 2) as avg_price, view from sales group by view order by cast(view as int) DESC) where avg_price>=350000""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.1719980239868164 seconds ---


In [43]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")

DataFrame[]

In [44]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [45]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""select * from (select round(avg(price), 2) as avg_price, view from sales group by view order by cast(view as int) DESC) where avg_price>=350000""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+----+
| avg_price|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.16099858283996582 seconds ---


In [46]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("sales_partition")

In [47]:
# 11. Read the formatted parquet data.
pDF = spark.read.parquet('sales_partition')

In [48]:
# 12. Create a temporary table for the parquet data.
pDF.createOrReplaceTempView('pSales')

In [49]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""select * from (select round(avg(price), 2) as avg_price, view from pSales group by view order by cast(view as int) DESC) where avg_price>=350000""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.47852444648742676 seconds ---


Uncached: 0.1719980239868164 seconds \
Cached: 0.16099858283996582 seconds \
Parquet: 0.47852444648742676 seconds

In [50]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [51]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("sales")

False